In [1]:
import pandas as pd
import numpy as np
import datetime
import time

In [2]:
# 读取数据集
df1 = pd.read_excel('./示例数据.xlsx',sheet_name='data1')
df1

,d,uid,action,active_duration
0,2021-01-01,1,jobs,324
1,2021-01-01,2,feeds,445
2,2021-01-01,3,im,345
3,2021-01-02,2,network,765
4,2021-01-02,3,feeds,342


In [3]:
# 读取数据集
df2 = pd.read_excel('./示例数据.xlsx',sheet_name='data2')
df2

,uid,career_level,city,work_length
0,1,中级,北京,4
1,2,高级,上海,7
2,3,初级,北京,1
3,4,普通,平顶山,2


In [4]:
# 初级
# 一月中，每日活跃的用户数
df = df1[['d','uid']].drop_duplicates()
con1 = df['d'] >= '2021/1/1'
con2 = df['d'] <= '2021/1/31'
df = df[con1 & con2]
df = df.groupby('d')['uid'].count().reset_index()
df

,d,uid
0,2021-01-01,3
1,2021-01-02,2


In [5]:
# 初级
# 一月中，对于每日活跃用户，其当日活跃时长最大的模块
con1 = df1['d'] >= '2021/1/1'
con2 = df1['d'] <= '2021/1/31'
df = df1[con1 & con2]
df['组内降序排名'] = df.groupby(['d'])['active_duration'].rank(method='dense',ascending=False).astype(int)
df = df[df['组内降序排名'] == 1]
df = df[['d','action']]
df

,d,action
1,2021-01-01,feeds
3,2021-01-02,network


In [6]:
# 初级
# 元旦期间的活跃用户中，jobs 模块活跃超过 100s 的用户
con1 = df1['d'] >= '2021/1/1'
con2 = df1['d'] <= '2021/1/3'
con3 = df1['action'] == 'jobs'
con4 = df1['active_duration'] > 100
df = df1[con1 & con2 & con3 & con4]
df = df['uid'].drop_duplicates().reset_index()
df

,index,uid
0,0,1


In [7]:
# 中级
# 昨日的活跃用户中，各个级别人才，每日最大活跃时长模块的活跃时长，占当日总活跃时长的比值的均值
today = '2021/1/3'
yesterday = (pd.to_datetime(today) - datetime.timedelta(days=1)).strftime('%Y/%m/%d')
df = df1[df1['d'] == yesterday]
df = pd.merge(df,df2,on='uid',how='left')
df = df.groupby(['career_level','action'])['active_duration'].sum().reset_index()
df = df.sort_values(by=['active_duration'])
df['占比'] = df['active_duration'] / df['active_duration'].sum()
df['组内降序排名'] = df.groupby(['career_level'])['active_duration'].rank(method='dense',ascending=False).astype(int)
mean_value = str(int(round(df[df['组内降序排名'] == 1]['占比'].mean() * 100,2))) + '%'
mean_value

'50%'

In [8]:
# 中级
# 过去一周内，北京用户的模块活跃总时长的均值
today = '2021/1/7'
begin_date = (pd.to_datetime(today) - datetime.timedelta(days=7)).strftime('%Y/%m/%d')
end_date = (pd.to_datetime(today) - datetime.timedelta(days=1)).strftime('%Y/%m/%d')
df = pd.merge(df1,df2,on='uid',how='left')
df = df[['d','city','action','active_duration']]
con1 = df['d'] >= begin_date
con2 = df['d'] <= end_date
con3 = df['city'] == '北京'
df = df[con1 & con2 & con3]
df = df.groupby(['d'])['active_duration'].sum().reset_index()
mean_value = df['active_duration'].mean()
mean_value

505.5

In [9]:
# 高级
# 过去一个月内，曾连续 2 天活跃的用户
today = '2021/1/30'
begin_date = (pd.to_datetime(today) - datetime.timedelta(days=30)).strftime('%Y/%m/%d')
end_date = (pd.to_datetime(today) - datetime.timedelta(days=1)).strftime('%Y/%m/%d')
con1 = df1['d'] >= begin_date
con2 = df1['d'] <= end_date
df = df1[con1 & con2]
df = df[['d','uid']].drop_duplicates()
df['rank'] = df['d'].groupby(df['uid']).rank(method='dense',ascending=True).astype(int)
df = df.sort_values(by=['uid','d'])
df['days'] = df['rank'].map(lambda x:pd.Timedelta(days=x))
df['minus'] = df['d'] - df['days']
df_group1 = df.groupby(['uid','minus'])['rank'].max().reset_index()
df_group2 = df_group1.groupby('uid')['rank'].max().reset_index()
df = df_group2[df_group2['rank'] == 2]
df

,uid,rank
1,2,2
2,3,2


In [10]:
# 高级
# 过去一个月内，不同级别用户的活跃频次差异
today = '2021/1/30'
begin_date = (pd.to_datetime(today) - datetime.timedelta(days=30)).strftime('%Y/%m/%d')
end_date = (pd.to_datetime(today) - datetime.timedelta(days=1)).strftime('%Y/%m/%d')
con1 = df1['d'] >= begin_date
con2 = df1['d'] <= end_date
df = df1[con1 & con2]
df = pd.merge(df1,df2,on='uid',how='left')
df = df[['d','career_level','uid','action','active_duration']]
df = df.groupby(['career_level'])['uid'].count().reset_index()
df

,career_level,uid
0,中级,1
1,初级,2
2,高级,2
